In [ ]:
# Various imports
import serial, threading, sys, time, os, datetime, warnings, numpy as np, matplotlib.pyplot as plt
from IPython.display import clear_output
from timeit import default_timer as timer
# The important imports
sys.path.append('/home/pi/Desktop/ADRNX/SRS')
import SRS

In [ ]:
# Initialize mainframe controller (with config file and optional debug)
mf = SRS.SIM900.SIM900('/home/pi/Desktop/ADRNX/SRS/config.yaml',debug=False)
mf.init_mainframe()
# Create module objects and add them to mainframe
s921 = SRS.SIM921.SIM921(mf,'s921',debug=False)
s922 = SRS.SIM922.SIM922(mf,'s922',debug=False)
s960 = SRS.SIM960.SIM960(mf,'s960',debug=False)
s970 = SRS.SIM970.SIM970(mf,'s970',debug=False)
mf.register_and_init_mods([s921, s922, s960, s970])
# Finally, start communication threads (this will enable all the queues)
mf.start_comm_threads()

In [ ]:
s922.send_cmd('CURV 0,1\n')

In [ ]:
s970.tellastory() # Essentially a __str__ function, gives summary of current state, a good check

In [ ]:
s922.tellastory()

In [ ]:
s921.tellastory()

In [ ]:
s960.tellastory()

In [ ]:
# Logic used to prevent unintended explosions, should be self explanatory
if abs(s960.get_output_voltage())>0.01: warnings.warn("Current is not zero - BE CAREFUL")
if s960.get_ramp_state() != 0: raise Exception("2")
if s960.get_PID_mode() == 0:
    if abs(s960.get_manual_output())>0.01: warnings.warn("Manual output is not zero - BE CAREFUL")
else:
    raise Exception("3")
if s960.get_offset_onoff() != 0: 
    if s960.get_offset != 0.0: raise Exception("4")
        
s960.set_PID_mode(0); assert s960.get_PID_mode() == 0
s960.set_offset(0); assert s960.get_offset() == 0
s960.set_offset_onoff(0); assert s960.get_offset_onoff() == 0

s960.set_setpoint(0); assert s960.get_setpoint() == 0
s960.set_setpoint_mode(0); assert s960.get_setpoint_mode() == 0
s960.set_ramp_onoff(0); assert s960.get_ramp_onoff() == 0
    
# Setting limits both in SIM960 and also in code as extra protection
ul = 2.5
s960.set_upper_lim(ul-0.1); assert s960.get_upper_lim() == ul-0.1
s960.manlim_high = ul

ll = -0.1
s960.set_lower_lim(-0.1); assert s960.get_lower_lim() == -0.1
s960.manlim_low = -0.1

s960.tellastory()

In [39]:
target = 2.0
maxemf = 0.150
vstep = 0.001 #Max slew rate in V/s

emfs, vouts, times, times2 = ([] for i in range(4))

vout = s960.get_manual_output()
vout2 = s960.get_manual_output()
assert vout == vout2
mf.clear_virtual_queues()
mf.clear_hw_queues()
if vout > target: 
    dir_up = 0 #down
elif vout == target: 
    warnings.warn("Target same as current value")
else:
    dir_up = 1 #up
delta = abs(vout - target)
s = timer()
print("Starting run - direction {}".format(dir_up))
barv = make_bar(min(target,vout),max(target,vout),prefix='Voltage: ',postfix='')
baremf = make_bar(0,300,prefix='Back EMF: ',postfix='mV')

i = 0
try:
    # Start streaming EMF channel
    s970.start_data_stream(SRS.SIM970.CHANNELS.MAGEMF)
    while(abs(vout - target) > 0.0):
        start_t = timer()
        gotevent = s970.await_next_event(tm=2.0,clear_before=True,clear_after=False)
        if not gotevent:
            raise RuntimeError("DID NOT GET VOLTAGE VALUE IN TIME - ABORTING")
        emf = abs(s970.lastvalues[SRS.SIM970.CHANNELS.MAGEMF])
        emfs.append(emf)
        update_bar(baremf,emf*1000)
        times.append(timer()-s)
        #print("EMF: {}".format(emf))    
        while (emf > maxemf): 
            # Loop until under threshold
            gotevent = s970.await_next_event(tm=1.5,clear_before=True,clear_after=False)
            if not gotevent:
                raise RuntimeError("DID NOT GET VOLTAGE VALUE IN TIME - ABORTING")            
            emf = abs(s970.lastvalues[SRS.SIM970.CHANNELS.MAGEMF]) 
            update_bar(baremf,emf*1000)
            #print("EMFW: {}".format(emf))
            
        # Set new manual voltage
        if (dir_up):
            if (vout + vstep < target): vnew = vout + vstep               
            else: vnew = target                   
        else:
            if (vout - vstep > target): vnew = vout - vstep                
            else: vnew = target                 
        s960.set_manual_output(vnew)
        time.sleep(0.2)
        
        # Check that new manual voltage is as intended (setting and actual output)
        attempts = 0
        while attempts < 2:
            try:
                vout_chk = s960.get_manual_output()
                time.sleep(0.2)  
                break
            except Exception as e: 
                print(e)
                attempts += 1
                warnings.warn("VMAN FAILED")
        
        attempts = 0
        while attempts < 2:
            try:            
                vmeas_chk = s960.get_output_voltage()
                time.sleep(0.2)  
                break
            except Exception as e: 
                print(e)
                attempts += 1
                warnings.warn("VMEAS FAILED")
        #clear_output()    
        print("{:05.2f}% | VMEAS: {:.5f} | VRESP: {:.5f} | VOLD: {:.5f} | VNEW:{:.5f} | EMF: {:.6f}".format(\
            100-abs(vout - target)*100/delta,vmeas_chk,vout_chk,vout,vnew,emf))
        vout = vnew; vouts.append(vmeas_chk); times2.append(timer()-s)
        try:
            warnings.warn(s960.excq.get_nowait())
            raise RuntimeError("Got s960 exc")
        except:
            pass
        assert abs(vout_chk - vnew) < 0.000001 #rounding of floats a problem 
        assert abs(vmeas_chk - vnew) < 0.01
        update_bar(barv,vout_chk)
        i+=1
        #print(i)
#         if i%20 == 0:
#             clear_output()
        # Wait remainder of 1 second
        while(timer()-start_t < 1.0): time.sleep(0.05)  
    print("DONE - took {:2f} minutes".format(timer()-s))
finally:
    # Stop data and clean up
    s970.stop_data_stream()
    time.sleep(0.2)
    s970.flush_queue()   

Starting run - direction 1


00.00% | VMEAS: 0.77069 | VRESP: 0.77000 | VOLD: 0.76900 | VNEW:0.77000 | EMF: 0.000012
00.08% | VMEAS: 0.77223 | VRESP: 0.77100 | VOLD: 0.77000 | VNEW:0.77100 | EMF: 0.000011
00.16% | VMEAS: 0.77326 | VRESP: 0.77200 | VOLD: 0.77100 | VNEW:0.77200 | EMF: 0.000012
00.24% | VMEAS: 0.77424 | VRESP: 0.77300 | VOLD: 0.77200 | VNEW:0.77300 | EMF: 0.000012
00.32% | VMEAS: 0.77423 | VRESP: 0.77400 | VOLD: 0.77300 | VNEW:0.77400 | EMF: 0.000012
00.41% | VMEAS: 0.77623 | VRESP: 0.77500 | VOLD: 0.77400 | VNEW:0.77500 | EMF: 0.000012
00.49% | VMEAS: 0.77725 | VRESP: 0.77600 | VOLD: 0.77500 | VNEW:0.77600 | EMF: 0.000011
00.57% | VMEAS: 0.77824 | VRESP: 0.77700 | VOLD: 0.77600 | VNEW:0.77700 | EMF: 0.000013
00.65% | VMEAS: 0.77925 | VRESP: 0.77800 | VOLD: 0.77700 | VNEW:0.77800 | EMF: 0.000014
00.73% | VMEAS: 0.77924 | VRESP: 0.77900 | VOLD: 0.77800 | VNEW:0.77900 | EMF: 0.000016
00.81% | VMEAS: 0.78129 | VRESP: 0.78000 | VOLD: 0.77900 | VNEW:0.78000 | EMF: 0.000016
00.89% | VMEAS: 0.78227 | VRESP:

07.64% | VMEAS: 0.86552 | VRESP: 0.86400 | VOLD: 0.86300 | VNEW:0.86400 | EMF: 0.000015
07.72% | VMEAS: 0.86550 | VRESP: 0.86500 | VOLD: 0.86400 | VNEW:0.86500 | EMF: 0.000017
07.80% | VMEAS: 0.86652 | VRESP: 0.86600 | VOLD: 0.86500 | VNEW:0.86600 | EMF: 0.000017
07.88% | VMEAS: 0.86851 | VRESP: 0.86700 | VOLD: 0.86600 | VNEW:0.86700 | EMF: 0.000014
07.96% | VMEAS: 0.86954 | VRESP: 0.86800 | VOLD: 0.86700 | VNEW:0.86800 | EMF: 0.000013
08.04% | VMEAS: 0.87053 | VRESP: 0.86900 | VOLD: 0.86800 | VNEW:0.86900 | EMF: 0.000014
08.12% | VMEAS: 0.87054 | VRESP: 0.87000 | VOLD: 0.86900 | VNEW:0.87000 | EMF: 0.000014
08.20% | VMEAS: 0.87257 | VRESP: 0.87100 | VOLD: 0.87000 | VNEW:0.87100 | EMF: 0.000014
08.29% | VMEAS: 0.87256 | VRESP: 0.87200 | VOLD: 0.87100 | VNEW:0.87200 | EMF: 0.000014
08.37% | VMEAS: 0.87358 | VRESP: 0.87300 | VOLD: 0.87200 | VNEW:0.87300 | EMF: 0.000012
08.45% | VMEAS: 0.87558 | VRESP: 0.87400 | VOLD: 0.87300 | VNEW:0.87400 | EMF: 0.000011
08.53% | VMEAS: 0.87661 | VRESP:

15.19% | VMEAS: 0.95746 | VRESP: 0.95700 | VOLD: 0.95600 | VNEW:0.95700 | EMF: 0.000008
15.27% | VMEAS: 0.95948 | VRESP: 0.95800 | VOLD: 0.95700 | VNEW:0.95800 | EMF: 0.000007
15.35% | VMEAS: 0.95949 | VRESP: 0.95900 | VOLD: 0.95800 | VNEW:0.95900 | EMF: 0.000006
15.43% | VMEAS: 0.96146 | VRESP: 0.96000 | VOLD: 0.95900 | VNEW:0.96000 | EMF: 0.000006
15.52% | VMEAS: 0.96251 | VRESP: 0.96100 | VOLD: 0.96000 | VNEW:0.96100 | EMF: 0.000007
15.60% | VMEAS: 0.96353 | VRESP: 0.96200 | VOLD: 0.96100 | VNEW:0.96200 | EMF: 0.000007
15.68% | VMEAS: 0.96452 | VRESP: 0.96300 | VOLD: 0.96200 | VNEW:0.96300 | EMF: 0.000007
15.76% | VMEAS: 0.96451 | VRESP: 0.96400 | VOLD: 0.96300 | VNEW:0.96400 | EMF: 0.000007
15.84% | VMEAS: 0.96655 | VRESP: 0.96500 | VOLD: 0.96400 | VNEW:0.96500 | EMF: 0.000006
15.92% | VMEAS: 0.96750 | VRESP: 0.96600 | VOLD: 0.96500 | VNEW:0.96600 | EMF: 0.000007
16.00% | VMEAS: 0.96850 | VRESP: 0.96700 | VOLD: 0.96600 | VNEW:0.96700 | EMF: 0.000007
16.08% | VMEAS: 0.96852 | VRESP:

ValueError: MANUAL OUTPUT 1.0000000000000002 OUT OF LIMITS

/home/pi/Desktop/ADRNX/SRS/SIM900.py:274: UserWarning: Bad message payload length - msg:|'MSG 3,#251StanforResearch_Systems,M960,s/n014669,ver17\rMS01.000431\r'| len:|57| (want |51|)
  warnings.warn("Bad message payload length - msg:|{}| len:|{}| (want |{}|)".format(repr(msg),len(msg3),numchar))
Exception in thread srsinput:
Traceback (most recent call last):
  File "/usr/lib/python3.4/threading.py", line 920, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.4/threading.py", line 868, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pi/Desktop/ADRNX/SRS/SIM900.py", line 176, in _input_loop
    self._iq_process(ser)
  File "/home/pi/Desktop/ADRNX/SRS/SIM900.py", line 221, in _iq_process
    [source,message] = self._msg_parser(resp)
  File "/home/pi/Desktop/ADRNX/SRS/SIM900.py", line 259, in _msg_parser
    numchar = int(msg2[2:4])#numchar = int(msg2[2:4])-2
ValueError: invalid literal for int() with base 10: '5t'



In [ ]:
s970.start_data_stream()

In [36]:
def make_bar(minv=0,maxv=1,prefix='Voltage: ',postfix='/2.0V'):
    from ipywidgets import FloatProgress,IntProgress, HTML, VBox
    from IPython.display import display

    progress = FloatProgress(min=minv,max=maxv,step=(maxv-minv)/1000)
    progress.bar_style = 'success'
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    label.value = prefix+''+postfix
    return [progress, label, box,prefix,postfix]
#     time.sleep(5)
#     progress.bar_style = 'danger'
#     time.sleep(5)
#     progress.bar_style = 'success'
    
def update_bar(bar,val,fmt='{:.3f}'):
    bar[0].value = val
    bar[1].value = bar[3]+fmt.format(val)+bar[4]

In [35]:
bar = make_bar(0,1.5,prefix='Voltage: ',postfix='/2.0V')
for i in np.arange(0,1,0.1):
    time.sleep(1)
    update_bar(bar,i)
    print('test')

test
test
test
test
test
test
test
test
test
test


In [ ]:
s970.stop_data_stream()
s921.stop_data_stream()
s922.stop_data_stream()

In [ ]:
#%matplotlib notebook

In [ ]:
fig = plt.figure()
plt.plot(times,emfs,'-')
plt.ylabel("EMF (V)")
plt.show()

fig = plt.figure()
plt.plot(times2,vouts,'-')
plt.ylabel("OUTPUT (V)")
plt.show()

In [ ]:
try:
    warnings.warn(s960.excq.get_nowait())
    raise RuntimeError("Got s960 exc")
except:
    pass